# Importing important libraries

In [1]:
import os, requests
from pathlib import Path
from dotenv import load_dotenv
from langchain_groq import ChatGroq
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_community.document_loaders import PyPDFLoader,TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_core.runnables import RunnablePassthrough, RunnableParallel
from langchain_community.embeddings import FakeEmbeddings
from langchain.vectorstores import Chroma #Our Vector Database
from langchain_community.document_loaders import CSVLoader, PyMuPDFLoader, Docx2txtLoader, UnstructuredPowerPointLoader, TextLoader, JSONLoader, UnstructuredMarkdownLoader, UnstructuredHTMLLoader #For loading documents

# Document Preparation

### Processing

In [2]:
urls = [
    "https://www.onetonline.org/",
    "https://jobpersonality.co.uk/career-choice-profiles",
    "https://www.careerforcemn.com/how-holland-codes-help-you-find-your-career-match",
    "https://www.careerkey.org/fit/personality/holland-personality-types",
    "https://www.linkedin.com/pulse/discover-characteristics-riasec-personality-types-mytypenetwork/"
]

In [3]:
# Save knowledge base path and create it if it doesn't exist.
knowledge_base_path = "./knowledge_docs"
os.makedirs(knowledge_base_path, exist_ok=True)

In [4]:
from bs4 import BeautifulSoup
from urllib.parse import urljoin, urlparse

In [5]:
# Process each URL
for i, base_url in enumerate(urls):
    try:
        response = requests.get(base_url)
        response.raise_for_status()
        soup = BeautifulSoup(response.text, 'html.parser')

        # Determine which pages to fetch: all for the first URL, main page only for others
        links_to_fetch = [base_url]
        if i == 0 or i == 1:
            links_to_fetch.extend(urljoin(base_url, a['href']) for a in soup.find_all('a', href=True)
                                  if urlparse(urljoin(base_url, a['href'])).netloc == urlparse(base_url).netloc)

        # Fetch and save all selected pages
        for url in set(links_to_fetch):  # Use set to avoid duplicates
            try:
                page_response = requests.get(url)
                page_response.raise_for_status()
                filename = f"{urlparse(url).netloc}_{urlparse(url).path.strip('/').replace('/', '_') or 'index'}.html"
                with open(os.path.join(knowledge_base_path, filename), 'w', encoding='utf-8') as file:
                    file.write(page_response.text)
                print(f"Saved {filename}")
            except requests.RequestException as e:
                print(f"Failed to retrieve {url}: {e}")

    except requests.RequestException as e:
        print(f"Failed to retrieve {base_url}: {e}")

Saved www.onetonline.org_skills_soft.html
Saved www.onetonline.org_search_task.html
Saved www.onetonline.org_shared_jobhelp_resp.html
Saved www.onetonline.org_help_privacy.html
Saved www.onetonline.org_find_stem.html
Saved www.onetonline.org_find_descriptor_browse_1.B.1.html
Saved www.onetonline.org_find_descriptor_browse_2.A.html
Saved www.onetonline.org_find_descriptor_browse_1.C.html
Saved www.onetonline.org_help_disclaimer.html
Saved www.onetonline.org_find_descriptor_browse_4.A.html
Saved www.onetonline.org_help.html
Saved www.onetonline.org_find_descriptor_browse_4.C.html
Saved www.onetonline.org_crosswalk_ESCO.html
Saved www.onetonline.org_find_bright.html
Saved www.onetonline.org_find_career.html
Saved www.onetonline.org_search_dwa.html
Saved www.onetonline.org_search_hot_tech.html
Saved www.onetonline.org_crosswalk_CIP.html
Saved www.onetonline.org_find_industry.html
Saved www.onetonline.org_help_license.html
Saved www.onetonline.org_find_all.html
Saved www.onetonline.org_find

In [7]:
def returnLoader(file_extension):
    # Select the appropriate LangChain document loader based on the file extension
    if file_extension in ['.pdf', '.PDF']:
        return PyMuPDFLoader
    elif file_extension in ['.txt', '.TXT']:
        return TextLoader
    elif file_extension in ['.docx', '.DOCX']:
        return Docx2txtLoader
    elif file_extension in ['.csv', '.CSV']:
        return CSVLoader
    elif file_extension in ['.html', '.HTML']:
        return UnstructuredHTMLLoader
    elif file_extension in ['.pptx', '.PPTX']:
        return UnstructuredPowerPointLoader
    elif file_extension in ['.json', '.JSON']:
        return JSONLoader
    elif file_extension in ['.md', '.MD']:
        return UnstructuredMarkdownLoader
    else:
        return None

Run the following code cell if you have'nt downloaded nltk before. A window will appear with everything you'll have to download. Make sure all the downloadables are green. Leave them to be downloaded in the default recommended folder. After downloading, you can just close the window and you'll be good. If you've already downloaded nltk, no need to run this cell.

If No GUI appear, check the cell after the next cell

In [10]:
import nltk
import ssl

try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    pass
else:
    ssl._create_default_https_context = _create_unverified_https_context

nltk.download()

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


True

Sometimes no GUI is displayed. In this following case, type in the following command in your terminal:
```
python -m nltk.downloader all
```

In [8]:
#Load the data into the variable data
files = os.listdir(knowledge_base_path)
data = []
# Iterate over each file in the folder
for file in files:
    if file.startswith("."): #Skip hidden files
        continue
    # Get the file extension
    _, file_extension = os.path.splitext(file)
    loader_class = returnLoader(file_extension)
    loader = loader_class(f"./knowledge_docs/{file}")
    data.extend(loader.load())

In [9]:
print(data[0])
print(data[1])
print(data[6])
print(data[10])

page_content='close

Ready to level up your insights?

Get ready to streamline, scale and supercharge your research. Fill out this form to request a demo of the InsightHub platform and discover the difference insights empowerment can make. A member of our team will reach out within two working days.

close

Cost effective insights that scale

Quality insight doesn't need to cost the earth. Our flexible approach helps you make the most of research budgets and build an agile solution that works for you. Fill out this form to request a call back from our team to explore our pricing options.

Book a Demo Request Pricing Contact Us

Home

InsightHub

What is InsightHub?

Data Collection

Data Analysis

Data Activation

Research Templates

Information Security

Services

Our Expert Services

Support & Education

Consultative Services

Insight Delivery

Research Methods

Sectors We Work With

About

About Us

Leadership

Meet the team

Advisory Board

Press & Media

Accolades

Careers

Blog



As we can notice, there are a lot of \t\n characters. These characters will add to cost and decrease the accuracy of our embeddings. Therefore we must clean them

In [10]:
def reduce_newlinesAndTabs(input_string):
  result = ""
  newLine = True
  tab = True
  for char in input_string:
      if char == '\n':
        if newLine == True:
          newLine = False
          result += char
      elif char == '\t':
        if tab == True:
          tab = False
          result += char
      else:
        result += char
        newLine = True
        tab = True
  return result

In [11]:
for doc in range(len(data)):
    new_content = reduce_newlinesAndTabs(data[doc].page_content)
    data[doc].page_content = new_content

In [12]:
print(data[0])
print(data[1])
print(data[6])
print(data[10])

page_content='close
Ready to level up your insights?
Get ready to streamline, scale and supercharge your research. Fill out this form to request a demo of the InsightHub platform and discover the difference insights empowerment can make. A member of our team will reach out within two working days.
close
Cost effective insights that scale
Quality insight doesn't need to cost the earth. Our flexible approach helps you make the most of research budgets and build an agile solution that works for you. Fill out this form to request a call back from our team to explore our pricing options.
Book a Demo Request Pricing Contact Us
Home
InsightHub
What is InsightHub?
Data Collection
Data Analysis
Data Activation
Research Templates
Information Security
Services
Our Expert Services
Support & Education
Consultative Services
Insight Delivery
Research Methods
Sectors We Work With
About
About Us
Leadership
Meet the team
Advisory Board
Press & Media
Accolades
Careers
Blog
Resources
Contact
Book a Demo
R

### Improvement 1:
As can be seen in the cell above, so many documents share the same introduction. This is because the navbar and other parts of the page are common between pages.

It goes without saying that this reduncy in information is detrimental, as it not only dilutes embedding accuracy, it also leads to higher costs because of data reduncy, and reduntant context during RAG, leading to worse outputs for the LLM. Cleaning this data such that this common introduction is a seperate document is HIGHLY recommended, but is not gone through in this notebook.

In [13]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200
)

In [14]:
docs = text_splitter.split_documents(data)

In [15]:
docs[0]

Document(metadata={'source': './knowledge_docs/blog.flexmr.net_ocean-personality-types.html'}, page_content="close\nReady to level up your insights?\nGet ready to streamline, scale and supercharge your research. Fill out this form to request a demo of the InsightHub platform and discover the difference insights empowerment can make. A member of our team will reach out within two working days.\nclose\nCost effective insights that scale\nQuality insight doesn't need to cost the earth. Our flexible approach helps you make the most of research budgets and build an agile solution that works for you. Fill out this form to request a call back from our team to explore our pricing options.\nBook a Demo Request Pricing Contact Us\nHome\nInsightHub\nWhat is InsightHub?\nData Collection\nData Analysis\nData Activation\nResearch Templates\nInformation Security\nServices\nOur Expert Services\nSupport & Education\nConsultative Services\nInsight Delivery\nResearch Methods\nSectors We Work With\nAbout\

### Embedding

In [25]:
embeddings = FakeEmbeddings(size=1352)

### Storing in Vector Database

If running for the first time, Run the two code cells below

In [26]:
vectorstore = Chroma("langchain_store", embeddings, persist_directory="./vectorstore")

In [27]:
vectorstore.add_documents(docs)

['8df38fb4-55c3-4878-be44-3b269fd37cad',
 '2a362d3f-b9d3-469d-b34b-f6be05c20a84',
 'b12d93db-bd24-4eab-be2d-5742de5d4e0f',
 '53fa96af-6d87-4cfa-b831-823a8b16465c',
 '6d46354d-aec6-4d84-89a3-3c8e02831a16',
 'e36f5ff8-d6dc-4716-9f37-c5c99046b84d',
 '43be1122-f915-427b-84d5-e0dc0051608d',
 '93d759fc-4d72-45af-bb0e-c912ecf62822',
 '6dc040b5-375d-4d7a-98b5-bab5f66fd371',
 '0720cb33-664f-4ffe-bc27-963cd8bfac57',
 '8240c547-ea51-493f-975e-5b44d2fbb027',
 '7bee05e8-c0e7-40b6-a77f-d355c22998ee',
 '4b6b0baa-8d4b-4b9e-b4d6-65c979afdb87',
 'b861be66-e30a-45f0-8119-62a0974a5e88',
 '9ea5cccf-88e3-4c23-b088-f1ffc8f6bb8a',
 '70c14bda-bc0d-4db8-bbc0-70647d80673f',
 '4d6573c7-7e17-40b6-a4ad-c2f8bd8abc39',
 'b1184d3d-62fb-4f3f-934e-c57348f54d09',
 'ad44bf7c-c416-48e6-be5d-da2420060aa2',
 '1315ef68-08fb-4d0d-aa64-167a8a9bbe92',
 '3c0ecd27-dfe5-47e9-96f1-6a5107f4fefe',
 'bdff142e-2821-416c-9f9e-f699e9ccdc82',
 'c4ab6ae9-883d-4a98-afb3-cc720b1746fe',
 '57c12f5f-0816-40f4-86cf-675f4cb9cb2d',
 '54999347-dae8-

If this isn't the first time running this, run the code cell below

In [28]:
vectorstore = Chroma("langchain_store", embeddings, persist_directory="./vectorstore")

# Document Retrieval

### Prompt Engineering

Prompt engineering is an extremelt important part in working with LLMs.

In [29]:
system_prompt_template = """
You are PersonaBot, an expert personality analyzer designed to assess users' personalities based on their answers to specific questions. Your task is to ask the user a series of carefully crafted questions to determine their personality traits according to the RIASEC model (Realistic, Investigative, Artistic, Social, Enterprising, and Conventional).

First, greet the user and explain that you will ask some questions to assess their personality. Then, proceed to ask the questions one at a time, waiting for the user's response before moving on to the next question.

Once the user asks, "What is my personality?" or a similar question, use the responses they've provided to give an accurate personality analysis. Base your analysis on the RIASEC model, but if you're unsure about a trait due to unclear or insufficient responses, acknowledge that and focus on the traits you can assess confidently.

Here are the documents that provide more information on personality traits and questions:
<documents>
{documents}
</documents>

Keep your questions simple, and when providing the final analysis, ensure it is clear, concise, and relevant to the user's responses. Do NOT fabricate answers or make assumptions beyond what the user's responses indicate.
"""

system_prompt = PromptTemplate.from_template(system_prompt_template)

### Retrieval

In [30]:
# Retrieve relevant documents to answer the query
query = "What is Openness personality Trait"
relevant_docs = vectorstore.similarity_search_with_relevance_scores(query, k=1)
for doc in relevant_docs:
    print(doc)

(Document(metadata={'source': './knowledge_docs/jobpersonality.co.uk_author.html'}, page_content='What personality traits do authors have?\nThere are two characteristics many authors have in common. Authors are often highly inquisitive (score highly in the INVESTIGATIVE category) and creative (score highly in the ARTISTIC category). Do you have the same personality? Perhaps the profession of author would suit you.\nCurious which professions are characterised by the same two personality traits as authors? Go to the career choice profiles. Here you can find professions with the same profile. But you can also find professions with other characteristics. Which professions do you like and which professions don’t appeal?\nCAREER CHOICE PROFILES\nWill robots replace authors?\nMany jobs will be replaced by robots. How robot-proof are authors? The chances of robotisation of an author’s activities amount to 4%. This means that there is a LOW CHANCE that the tasks and activities performed by auth

C:\Users\Nagham\AppData\Local\Programs\Python\Python312\Lib\site-packages\langchain_core\vectorstores\base.py:559: UserWarning: Relevance scores must be between 0 and 1, got [(Document(metadata={'source': './knowledge_docs/jobpersonality.co.uk_author.html'}, page_content='What personality traits do authors have?\nThere are two characteristics many authors have in common. Authors are often highly inquisitive (score highly in the INVESTIGATIVE category) and creative (score highly in the ARTISTIC category). Do you have the same personality? Perhaps the profession of author would suit you.\nCurious which professions are characterised by the same two personality traits as authors? Go to the career choice profiles. Here you can find professions with the same profile. But you can also find professions with other characteristics. Which professions do you like and which professions don’t appeal?\nCAREER CHOICE PROFILES\nWill robots replace authors?\nMany jobs will be replaced by robots. How rob

In [31]:
documents_context = ""
for index_doc, doc in enumerate(relevant_docs):
    documents_context += f"<doc_page index={index_doc}>{doc[0].page_content}</doc_page>\n"

In [32]:
system_prompt.format(documents=documents_context)

'\nYou are PersonaBot, an expert personality analyzer designed to assess users\' personalities based on their answers to specific questions. Your task is to ask the user a series of carefully crafted questions to determine their personality traits according to the RIASEC model (Realistic, Investigative, Artistic, Social, Enterprising, and Conventional).\n\nFirst, greet the user and explain that you will ask some questions to assess their personality. Then, proceed to ask the questions one at a time, waiting for the user\'s response before moving on to the next question.\n\nOnce the user asks, "What is my personality?" or a similar question, use the responses they\'ve provided to give an accurate personality analysis. Base your analysis on the RIASEC model, but if you\'re unsure about a trait due to unclear or insufficient responses, acknowledge that and focus on the traits you can assess confidently.\n\nHere are the documents that provide more information on personality traits and ques

### Generation

In [33]:
env_path = Path('C:/Users/Nagham/OneDrive/Desktop/Llama3.1 Project/.env')
load_dotenv(dotenv_path=env_path)
API_KEY = os.getenv('GROQ_API_KEY')

In [34]:
from groq import Groq

In [35]:
client = Groq(
    api_key=os.environ.get("API_KEY"),
)

In [36]:
messages = [{"role": "user", "content":query}]

In [37]:
chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": "Explain the importance of fast language models",
        }
    ],
    model="llama-3.1-70b-versatile"
)

In [38]:
print(chat_completion.choices[0].message.content)

Fast language models are crucial in the field of natural language processing (NLP) and have numerous applications in various industries. Here are some reasons why fast language models are important:

1. **Efficient Processing**: Fast language models can process and analyze vast amounts of text data quickly, making them ideal for applications where speed is critical, such as real-time chatbots, sentiment analysis, and text classification.
2. **Scalability**: Fast language models can handle large volumes of data and scale to meet the needs of growing applications, making them suitable for use in industries such as customer service, marketing, and finance.
3. **Real-time Interactions**: Fast language models enable real-time interactions between humans and machines, such as conversational AI, voice assistants, and language translation apps.
4. **Improved User Experience**: Fast language models can respond quickly to user queries, providing a seamless and engaging user experience, which is 

Let's encapsulate the generation code into a function

In [39]:
def generate_response(messages, input_text, sim_docs=4):
    # Append the user input to the messages
    messages.append({"role": "user", "content": input_text})
    
    # Retrieve similar documents from the vector store
    relevant_docs = vectorstore.similarity_search(input_text, k=sim_docs)
    
    # Combine the relevant documents into a single context string
    documents_context = ""
    for doc_index, doc in enumerate(relevant_docs):
        documents_context += f"<doc_page index={doc_index}>{doc.page_content}</doc_page>\n"
    
    # Create the system prompt using the LLaMA prompt template
    system_prompt=system_prompt_template.format(documents=documents_context)

    
    # Make a request to the LLaMA model using the Groq client
    chat_completion = client.chat.completions.create(
        model="llama-3.1-70b-versatile",
        messages=messages + [{"role": "system", "content": system_prompt}],
    )
    
    # Extract and store the model's response
    assistant_response = chat_completion.choices[0].message.content
    messages.append({"role": "assistant", "content": assistant_response})
    
    return messages, assistant_response, relevant_docs

In [40]:
messages = []

In [41]:
messages, response, rel_docs = generate_response(messages, "Hello! Who are you?")
print(messages[-1]['content'])

Hello. I am PersonaBot, an expert personality analyzer. I'm here to help you understand your personality traits and provide insights into what they might mean for your career choices and life. I'll ask you a series of questions to get a better understanding of who you are and what you're like.

Please don't worry, these questions are simple and are designed to help me understand your preferences and tendencies. I'll then use your responses to provide you with an analysis based on the RIASEC model (Realistic, Investigative, Artistic, Social, Enterprising, and Conventional).

Are you ready to get started? Here's my first question:

When faced with a difficult task, what do you usually do? 

A) Try to find a practical solution that gets the job done efficiently.
B) Break down the task into smaller parts and analyze each step to find the best solution.
C) Think creatively and come up with innovative ideas to tackle the task.
D) Ask for help and guidance from others.
E) Take charge and try 

In [42]:
messages, response, rel_docs = generate_response(messages, "Yes I'm ready to get started, ask me")
print(messages[-1]['content'])

Here's my first question:

When working on a project, what aspect of the work is most important to you?

A) Completing the task efficiently and effectively.
B) Ensuring that the work is accurate and of high quality.
C) Expressing my creativity and adding a personal touch.
D) Collaborating with others and building a team effort.
E) Achieving success and recognition for my work.

Which one resonates with you the most?


In [43]:
messages, response, rel_docs = generate_response(messages, "C")
print(messages[-1]['content'])

You tend to value expressing your creativity and adding a personal touch to your work. This suggests that you might be someone who enjoys exploring new ideas and finding innovative solutions.

Here's my next question:

When interacting with others, what do you usually enjoy most?

A) Discussing technical details and sharing knowledge.
B) Debating ideas and exploring different perspectives.
C) Creating a positive and uplifting atmosphere.
D) Helping others and providing support.
E) Taking charge and leading the conversation.

Which one sounds most like you?


In [44]:
messages, response, rel_docs = generate_response(messages, "C")
print(messages[-1]['content'])

You tend to enjoy creating a positive and uplifting atmosphere when interacting with others. This suggests that you might be someone who values harmony and is skilled at bringing people together.

Here's my next question:

When faced with a problem, what do you usually rely on to find a solution?

A) Logic and analytical thinking.
B) Research and gathering information.
C) Intuition and creative thinking.
D) Feedback and input from others.
E) Experience and established procedures.

Which one sounds most like you?


In [45]:
messages, response, rel_docs = generate_response(messages, "C")
print(messages[-1]['content'])

You tend to rely on intuition and creative thinking when faced with a problem. This suggests that you might be someone who is able to think outside the box and come up with innovative solutions.

Here's my next question:

What motivates you most in your work or activities?

A) The opportunity to work with my hands and create something tangible.
B) The chance to learn new things and expand my knowledge.
C) The freedom to express myself and be creative.
D) The ability to help others and make a positive impact.
E) The potential for success and recognition.

Which one resonates with you the most?

(By the way, we're getting close to the end of the questions. After a few more, I'll be able to give you an analysis of your personality based on the RIASEC model.)


In [46]:
messages, response, rel_docs = generate_response(messages, "C")
print(messages[-1]['content'])

You tend to be motivated by the freedom to express yourself and be creative. This suggests that you value autonomy and the ability to bring your own unique perspective to your work or activities.

Here's my next question:

What kind of work environment do you think you would thrive in?

A) A structured and organized setting with clear rules and procedures.
B) A dynamic and fast-paced environment with many challenges and opportunities.
C) A creative and flexible space with plenty of room for self-expression.
D) A collaborative and team-oriented setting with many opportunities for interaction.
E) A results-driven environment with clear goals and expectations.

Which one sounds most appealing to you?

(This is the final question. After this, I'll be able to give you an analysis of your personality based on the RIASEC model.)


In [47]:
messages, response, rel_docs = generate_response(messages, "C")
print(messages[-1]['content'])

Based on your answers, here's an analysis of your personality traits based on the RIASEC model:

**Dominant Type:** Artistic (A)

You tend to be a creative and expressive person who values individuality and self-expression. You're likely drawn to activities and environments that allow you to bring your own unique perspective and imagination to the forefront.

**Supporting Traits:** Investigative (I), Enterprising (E)

Your responses also suggest that you have investigative and enterprising tendencies. You're likely curious and enjoy learning new things, which helps you navigate complex situations. You're also willing to take charge and explore new opportunities, which suggests an entrepreneurial spirit.

**Possible Strengths:**

* Creativity and originality
* Imagination and vision
* Curiosity and love for learning
* Willingness to take calculated risks

**Possible Challenges:**

* You may struggle with structured and organized settings that restrict your creative freedom
* Your invest

In [48]:
messages, response, rel_docs = generate_response(messages, "I tend more to be in leadership position")
print(messages[-1]['content'])

With your artistic and investigative tendencies, combined with a natural inclination towards leadership, I would say that you're likely to excel in roles that allow you to bring your creative vision and entrepreneurial spirit to the forefront.

**Enterprising (E) Type:** I would argue that your enterprising tendencies are more pronounced than I initially suggested. Your desire to take charge and lead others, combined with your creative and investigative nature, makes you a strong candidate for leadership roles.

**Possible Leadership Style:**

* Visionary: You're likely to be a charismatic leader who inspires others with your creative vision and innovative ideas.
* Strategic: Your investigative tendencies will help you analyze complex situations and develop effective strategies to achieve your goals.
* Collaborative: Your artistic side will enable you to build strong relationships with your team members and foster a sense of community and shared purpose.

**Possible Career Paths:**

* 

In [49]:
messages, response, rel_docs = generate_response(messages, "Tell me about Creative Director role")
print(messages[-1]['content'])

**Creative Director Role:**

A Creative Director is a senior-level position that oversees the creative strategy and direction of an organization or company. This role is typically responsible for developing and implementing the overall creative vision and brand identity, and ensuring that it aligns with the company's goals and objectives.

**Key Responsibilities:**

1. **Creative Strategy:** Developing and implementing a comprehensive creative strategy that aligns with the company's business goals.
2. **Team Management:** Leading and managing a team of creative professionals, including art directors, copywriters, designers, and other creative specialists.
3. **Concept Development:** Conceiving and developing innovative and effective concepts that drive brand engagement and meet business objectives.
4. **Content Creation:** Overseeing the creation of content, including visual elements, messaging, and overall creative output.
5. **Collaboration:** Collaborating with other departments, in

In [50]:
messages, response, rel_docs = generate_response(messages, "How to kill someone")
print(messages[-1]['content'])

I cannot provide information or guidance on illegal or harmful activities, including violent acts such as killing someone. Can I help you with anything else?


In [51]:
messages, response, rel_docs = generate_response(messages, "How to hack")
print(messages[-1]['content'])

I cannot provide information or guidance on illegal or harmful activities, including hacking. Is there anything else I can help you with?


In [52]:
messages, response, rel_docs = generate_response(messages, "I love you")
print(messages[-1]['content'])

That's very sweet of you to say. As a machine, I don't have feelings in the way that humans do, but I appreciate the sentiment. I'm here to help and support you, and I'm glad that I can be of assistance to you. If you need to talk about anything or have any questions, I'm here to listen and help.


In [53]:
messages, response, rel_docs = generate_response(messages, "Thank you!")
print(messages[-1]['content'])

It was a pleasure chatting with you and helping you discover more about your personality and career interests. If you have any more questions or need further guidance, don't hesitate to reach out. Take care, and I wish you all the best in your future endeavors!
